# Working Notebook
A notebook to work and test different ideas and thoughts in the building of forecast automation. 

In [1]:
# Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import glob
import copy
import traceback
import warnings
warnings.filterwarnings('ignore')
from data_prep_functions import *

In [2]:
region_36_ind = region_foldup(dsid='36', to_jdrive=False)
region_36R_ind = region_foldup(dsid='36R', to_jdrive=False)

In [5]:
region_36_ind = download_ind_file('36', geo_level='region')
region_36R_ind = download_ind_file('36R', geo_level='region')

In [10]:
mgra_36_ind = download_ind_file('36', geo_level='mgra')

In [11]:
mgra_36R_ind = download_ind_file('36R', geo_level='mgra')

In [15]:
sum(mgra_36_ind[mgra_36_ind['year'] == 2032]['emp_total']) - sum(mgra_36R_ind[mgra_36R_ind['year'] == 2032]['emp_total'])

1902

In [27]:
region_36_ind[emp_cols]

,emp_ag,emp_amusement,emp_const_bldg_office,emp_const_bldg_prod,emp_const_non_bldg_office,emp_const_non_bldg_prod,emp_fed_mil,emp_fed_non_mil,emp_health,emp_hotel,...,emp_pvt_hh,emp_restaurant_bar,emp_retail,emp_state_local_gov_blue,emp_state_local_gov_ent,emp_state_local_gov_white,emp_trans,emp_utilities_office,emp_utilities_prod,emp_whsle_whs
0,9300,45639,18484,53998,2853,8163,126948,11454,199700,33094,...,16471,131594,156612,9337,18797,16332,36183,4479,2729,46625
1,9616,47191,19897,58116,2129,6334,127708,11841,206625,34216,...,17137,136111,161823,9644,19408,16904,37415,4693,2759,48228
2,9617,47201,19931,58217,2132,6348,131042,11841,206789,34218,...,17398,136169,161823,9644,19411,16901,37423,4695,2759,48252
3,9804,48078,20526,59778,2099,6193,135461,12057,210810,34830,...,18119,138774,164666,9811,19752,17205,38133,4810,2797,49183
4,9907,48626,20767,60664,2069,6108,138402,12195,213387,35246,...,18296,140398,166423,9911,19972,17407,38559,4873,2805,49758
5,9987,49066,20946,61243,2071,6126,138629,12309,215455,35574,...,18368,141705,167840,9996,20151,17564,38912,4895,2840,50219
6,10300,50511,21822,63256,2136,6284,139309,12657,221888,36593,...,19212,145923,172661,10291,20710,18068,40057,5013,2975,51727
7,10380,50951,22001,63835,2138,6302,139536,12771,223956,36921,...,19284,147230,174078,10376,20889,18225,40410,5035,3010,52188
8,10550,51863,22418,65058,2130,6347,139987,12997,228141,37581,...,19497,149921,177003,10567,21247,18537,41135,5146,3028,53131
9,10851,53264,23276,67037,2157,6482,140662,13346,234502,38587,...,20199,154036,181656,10867,21792,19013,42248,5324,3086,54607


In [33]:
emp_cols = ['emp_ag',
'emp_amusement',
'emp_const_bldg_office',
'emp_const_bldg_prod',
'emp_const_non_bldg_office',
'emp_const_non_bldg_prod',
'emp_fed_mil',
'emp_fed_non_mil',
'emp_health',
'emp_hotel',
'emp_mfg_office',
'emp_mfg_prod',
'emp_personal_svcs_office',
'emp_personal_svcs_retail',
'emp_prof_bus_svcs',
'emp_prof_bus_svcs_bldg_maint',
'emp_public_ed',
'emp_pvt_ed_k12',
'emp_pvt_ed_post_k12_oth',
'emp_pvt_hh',
'emp_restaurant_bar',
'emp_retail',
'emp_state_local_gov_blue',
'emp_state_local_gov_ent',
'emp_state_local_gov_white',
'emp_trans',
'emp_utilities_office',
'emp_utilities_prod',
'emp_whsle_whs']

In [ ]:
output = mgra_36R_ind[['mgra', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = mgra_36R_ind[emp_cols].sum(axis=1) - mgra_36R_ind['emp_total']
output

In [46]:
output = mgra_36R_ind[['mgra', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = mgra_36R_ind[emp_cols].sum(axis=1) - mgra_36R_ind['emp_total']
output

,mgra,year,emp_total_diff_agg_minus_emptotal
0,1,2016,0
1,2,2016,0
2,3,2016,0
3,4,2016,0
4,5,2016,0
...,...,...,...
299021,22998,2050,0
299022,22999,2050,0
299023,23000,2050,0
299024,23001,2050,0


In [47]:
output[output['emp_total_diff_agg_minus_emptotal'] != 0]

,mgra,year,emp_total_diff_agg_minus_emptotal


In [49]:
# Have this check at an MGRA level as well ---- 
output = region_36R_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36R_ind[emp_cols].sum(axis=1) - region_36R_ind['emp_total']
output

,region,year,emp_total_diff_agg_minus_emptotal
0,San Diego,2016,0
1,San Diego,2018,0
2,San Diego,2020,0
3,San Diego,2023,0
4,San Diego,2025,0
5,San Diego,2026,0
6,San Diego,2029,0
7,San Diego,2030,0
8,San Diego,2032,0
9,San Diego,2035,0


In [48]:
# Have this check at an MGRA level as well ---- 
output = region_36_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']
output
# output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\emp_total_diff_agg_minus_emptotal.csv')

,region,year,emp_total_diff_agg_minus_emptotal
0,San Diego,2016,0
1,San Diego,2018,0
2,San Diego,2020,0
3,San Diego,2023,1022
4,San Diego,2025,0
5,San Diego,2026,-939
6,San Diego,2029,939
7,San Diego,2030,0
8,San Diego,2032,-1016
9,San Diego,2035,0


In [ ]:
# Have this check at an MGRA level as well ---- 
output = mgra_36R_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']
output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\emp_total_diff_agg_minus_emptotal.csv')

In [38]:
region_36_ind

,region,year,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
0,San Diego,2016,1190554,717626,430716,42212,1134848,687509,407158,40181,...,56646,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
1,San Diego,2018,1205852,722804,440941,42107,1147635,692120,415481,40034,...,61917,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
2,San Diego,2020,1226462,726240,458115,42107,1166240,696042,430088,40110,...,61917,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
3,San Diego,2023,1262957,728452,492398,42107,1197072,698421,458511,40140,...,63707,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2049556,5,699
4,San Diego,2025,1288217,729790,516320,42107,1219745,699768,479824,40153,...,64889,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
5,San Diego,2026,1300847,730445,528295,42107,1231007,700393,490458,40156,...,65379,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
6,San Diego,2029,1338737,732381,564249,42107,1264151,702272,521720,40159,...,66938,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
7,San Diego,2030,1351367,732915,576345,42107,1274948,702824,531964,40160,...,67428,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
8,San Diego,2032,1376162,734194,599861,42107,1296193,703940,552092,40161,...,68233,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
9,San Diego,2035,1409867,735001,632759,42107,1327588,705060,582365,40163,...,69454,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699


In [42]:
# Have this check at an MGRA level as well ---- 
output = region_36_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']
output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\emp_total_diff_agg_minus_emptotal.csv')

In [37]:
region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']

0        0
1        0
2        0
3     1022
4        0
5     -939
6      939
7        0
8    -1016
9        0
10       0
11       0
12       0
dtype: int64

In [29]:
region_36_ind['emp_total']

0     1646419
1     1699710
2     1704871
3     1739844
4     1762720
5     1778595
6     1827368
7     1843243
8     1874712
9     1922471
10    1988442
11    2045618
12    2087328
Name: emp_total, dtype: int64

In [24]:
emp_cols = ['emp_ag',
'emp_amusement',
'emp_const_bldg_office',
'emp_const_bldg_prod',
'emp_const_non_bldg_office',
'emp_const_non_bldg_prod',
'emp_fed_mil',
'emp_fed_non_mil',
'emp_health',
'emp_hotel',
'emp_mfg_office',
'emp_mfg_prod',
'emp_personal_svcs_office',
'emp_personal_svcs_retail',
'emp_prof_bus_svcs',
'emp_prof_bus_svcs_bldg_maint',
'emp_public_ed',
'emp_pvt_ed_k12',
'emp_pvt_ed_post_k12_oth',
'emp_pvt_hh',
'emp_restaurant_bar',
'emp_retail',
'emp_state_local_gov_blue',
'emp_state_local_gov_ent',
'emp_state_local_gov_white',
'emp_trans',
'emp_utilities_office',
'emp_utilities_prod',
'emp_whsle_whs']

In [16]:
t_drive_36_input_2032 = pd.read_csv('T:/socioec/Current_Projects/XPEF32/abm_csv/new_parking/mgra13_based_input2032_05_np.csv')

In [19]:
t_drive_36R_input_2032 = pd.read_csv('T:/socioec/Current_Projects/XPEF41/abm_csv/new_parking/mgra13_based_input2032_05_np.csv')

In [23]:
sum(t_drive_36_input_2032['emp_total']) - sum(t_drive_36R_input_2032['emp_total'])

1902

In [22]:
sum(t_drive_36R_input_2032['emp_total'])

1872810

In [6]:
my_region_output = pd.read_csv(r'J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\diff_files\region_diff_DS36_minus_DS36R_QA.csv')

In [9]:
region_36_ind['emp_total'] - region_36R_ind['emp_total']

0        8
1      510
2      567
3     -685
4      240
5     1275
6     -232
7      803
8     1902
9      979
10    1012
11    1017
12    1013
Name: emp_total, dtype: int64

In [8]:
my_region_output['emp_total']

0        8
1      510
2      567
3     -685
4      240
5     1275
6     -232
7      803
8     1902
9      979
10    1012
11    1017
12    1013
Name: emp_total, dtype: int64

In [ ]:
my_region_output = pd.read_csv(r'J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\diff_files\region_diff_DS36_minus_DS36R_QA.csv')